In [1]:
from dotenv import load_dotenv
load_dotenv()

import os

from db.postgres.db_postgres import PostgresExecutor
from db.snowflake.db_snowflake import SnowflakeExecutor

db_params = {
    "host": 'host.docker.internal',
    "port": '5433',
    "username": 'admin',
    "password": 'admin',
    "db": 'crime_db',
}

snow_params = {
    "account" : 'QYKNAZY-GD18580',
    "database" : 'crime_db',
    "schema" : 'PUBLIC',
    "role" : 'ACCOUNTADMIN',
    "warehouse" : 'crime_wh',
    "user" : 'hyderreza',
    "password" : os.getenv("SNOWFLAKE_DB_PASSWORD")
}

post = PostgresExecutor(**db_params, template_path='../include/sql/postgres')
snow = SnowflakeExecutor(**snow_params, template_path='../include/sql/postgres')

/home/hyderreza/codehub/open-crime-etl-pipeline/airflow/.venv/lib/python3.10/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (21.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [22]:
params = {
    'run_id' : 'test',
    'load_date' : 'somedate',
    'status' : 'SUCCESS',
    'end_time' : '3:19'
}

table = 'logs'
where = ['run_id', 'load_date']

values = {k : v for k,v in params.items()}

set_values = ", ".join([f"{k} = :{k}" for k in values.keys() if k not in where])

missing_keys = [k for k in where if k not in values]
if missing_keys:
    raise ValueError(f"Missing primary key(s) in update: {missing_keys}")

where_clause = ' AND '.join([f"{k} = :{k}" for k in values.keys() if k in where])

f"UPDATE {table} SET {set_values} WHERE {where_clause}"


'UPDATE logs SET status = :status, end_time = :end_time WHERE run_id = :run_id AND load_date = :load_date'

In [19]:
values = {k : v for k,v in params.items()}

set_clause = ", ".join(params.keys())
val_clause = ", ".join([f':{k}' for k in values.keys()])

f"INSERT INTO {table}({set_clause}) VALUES ({val_clause})"

'INSERT INTO logs(run_id, load_date, status, end_time) VALUES (:run_id, :load_date, :status, :end_time)'

In [ ]:
from datetime import datetime, date
def date_to_str(date: date):
    return date.strftime("%Y-%m-%d")


In [27]:
import pandas as pd

query = "SELECT ingested_at FROM logs"
# with post.engine.begin() as conn:
#     r = conn.execute(query)
#     data1 = set(r.fetchall())
#     df1 = pd.DataFrame(r.mappings().fetchall())
    
with snow.engine.begin() as conn:
    r = conn.execute(query)
    data2 = list(set(r.fetchall()))
    df2 = pd.DataFrame(r.mappings().fetchall())
    print(data2)
    # print(type([date_to_str(d[0]) for d in data2]))

[(datetime.date(2025, 8, 18),)]


In [10]:
type(list(data1)[0][0])

datetime.date

In [61]:
d_a = {1,2,3}
d_b = {3,4,5}

miss_a = d_b - d_a # Answer should be 4,5
miss_b = d_a - d_b # Answer should be 1,2
print(miss_a)
print(miss_b)

{4, 5}
{1, 2}


In [54]:
d1 = pd.DataFrame({'col1': [1, 2, 3]})
d2 = pd.DataFrame({'col2': [3, 4, 5]})

result = pd.merge(d1,d2, left_on='col1', right_on='col2', how='outer')
missing_db1 = result[result['col1'].isnull()]['col2']
missing_db2 = result[result['col2'].isnull()]['col1']
result

,col1,col2
0,1.0,NaN
1,2.0,NaN
2,3.0,3.0
3,NaN,4.0
4,NaN,5.0


In [55]:
missing_db1

3    4.0
4    5.0
Name: col2, dtype: float64

In [56]:
missing_db2

0    1.0
1    2.0
Name: col1, dtype: float64

In [92]:
from datetime import timedelta, datetime
import re

single_date = True

last_load_date = datetime(2025,8,14).date()
today_date = datetime.now().date()
if not single_date:
    dates_to_process = [(last_load_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((today_date - last_load_date).days+1)]
else:
    dates_to_process = [last_load_date.strftime("%Y-%m-%d")]

load_dates = "|".join(map(re.escape, dates_to_process))
regex = re.compile(rf"^raw/year=\d{{4}}/month=\d{{2}}/load_date=({load_dates})/.*")
regex

re.compile(r'^raw/year=\d{4}/month=\d{2}/load_date=(2025\-08\-14)/.*',
           re.UNICODE)